In [70]:
############### Set working directory to parent directory
import os,  requests, sys, re, urllib.request, pandas as pd
from bs4 import BeautifulSoup

os.getcwd()
# os.chdir('F:\\github\\narrative_conservatism\\code')

'F:\\github\\narrative_conservatism\\code'

In [73]:
#################### Access all fillings through SEC master index #################################
####### indexes downloaded using python-edgar: https://github.com/edouardswiac/python-edgar #######
#### open terminal, and run the following lines UNDER THE REPO'S ROOT DIR:
#### python run.py -y 1993 -d edgar_idx (downloading all quarterly master index from 1993 into folder edgar_idx)
#### cd F:\github\python-edgar-master\edgar-idx (switch dir to where the downloaded indexes are located)
#### cat *.tsv > master.tsv (stitch all quarterly indexes into one master index)
#### du -h master.tsv (inspect how large the master index file is)

obj_type = '10-K'

doc_url = list()
# create an index of downloaded local quarterly master indexes
index_input = list()
for subdir, dirs, files in os.walk("F:\github\python-edgar-master\edgar-idx"):
    for file in files:
        index_input.append(os.path.join(subdir, file))

# read each index file, select rows with matched file type, and store matched doc_links
for filenameTSV in index_input:
    tsv_read = pd.read_csv(filenameTSV, sep='|', header=None, encoding = "utf-8")
    tsv_read.columns = ['1', '2', '3', '4', '5', '6']
    
    # select the rows with filetype equal to predefined type
    tsv_type = tsv_read.loc[tsv_read['3'] == obj_type]
    doc_link = tsv_type['6'].values.tolist()
    doc_link = ['https://www.sec.gov/Archives/' + w for w in doc_link]
    for doc in doc_link:
        doc_url.append(doc)
        
len(doc_url)

192755

In [69]:
# #################### Access all fillings through SEC search engine ####################################
# ################## NOT RECOMMENDED AT ALL #############################################################
# cik = '0000051143'
# obj_type = '8-K'
# number of documents listed per page
# count = '100'
# # index of first document listed in the current page
# start = '0'
# # find filings prior to the date 2016y01m01d
# dateb = ''

# # Obtain url for intial search result page
# base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}&start={}&count={}"
# init_url = base_url.format(cik, obj_type, dateb, start, count)

# # define a function that takes the input url and returns next search page url
# def get_next_url(input_url):
#     edgar_resp = requests.get(input_url)
#     edgar_str = edgar_resp.text
#     soup = BeautifulSoup(edgar_str, 'html.parser')

#     div_tag = soup.find('div', style='margin-top: 5px; margin-bottom: 5px;')
#     button = div_tag.find('td', style='text-align: right;')
#     fbutton = button.find_all('input')[0]['value']
#     if re.findall(r'Next', fbutton) == ['Next']:
#         next_url = button.find_all('input')[0]['onclick'][:-1]
#     elif len(button.find_all('input')) == 2:
#         next_url = button.find_all('input')[1]['onclick'][:-1]
#     else:
#         next_url = 'NA'
        
#     next_url = next_url.replace('parent.location=\'', 'https://www.sec.gov')
#     return next_url

# # create a search result page url list
# search_url = [init_url]

# while get_next_url(init_url) != 'NA':
#     search_url.append(get_next_url(init_url))
#     init_url = get_next_url(init_url)
    
# ############### Create a document link list of a given CIK and file type
# doc_link = list()

# for url in search_url:
#     edgar_resp = requests.get(url)
#     edgar_str = edgar_resp.text
#     soup = BeautifulSoup(edgar_str, 'html.parser')
#     table_tag = soup.find('table', class_='tableFile2')
#     rows = table_tag.find_all('tr')

#     for row in rows[1:]:
#         cells = row.find_all('td')
#         doc_link.append('https://www.sec.gov' + cells[1].a['href'])
        
# len(doc_link)

In [79]:
#############################################Some of the first years fillings do not have a html link, has to use '	Complete submission text file', drop????
doc_url[-20:]

['https://www.sec.gov/Archives/edgar/data/95552/0001564590-20-007814-index.html',
 'https://www.sec.gov/Archives/edgar/data/95574/0001437749-20-003101-index.html',
 'https://www.sec.gov/Archives/edgar/data/95953/0000095953-20-000021-index.html',
 'https://www.sec.gov/Archives/edgar/data/96223/0000096223-20-000008-index.html',
 'https://www.sec.gov/Archives/edgar/data/96271/0001564590-20-004873-index.html',
 'https://www.sec.gov/Archives/edgar/data/9631/0000891092-20-000862-index.html',
 'https://www.sec.gov/Archives/edgar/data/96869/0000096869-20-000007-index.html',
 'https://www.sec.gov/Archives/edgar/data/96943/0000096943-20-000017-index.html',
 'https://www.sec.gov/Archives/edgar/data/97134/0000097134-20-000008-index.html',
 'https://www.sec.gov/Archives/edgar/data/97210/0001193125-20-058676-index.html',
 'https://www.sec.gov/Archives/edgar/data/97216/0000097216-20-000027-index.html',
 'https://www.sec.gov/Archives/edgar/data/97476/0000097476-20-000009-index.html',
 'https://www.sec

In [87]:
############### Extract file identification info + HTML links
html_link = list()
accnum = list()
fd = list()
rp = list()
name = list()
cik = list()
sic = list()
filetype = list()
fye = list()
state = list()
bazip = list()
item8k = list()

for doc in doc_url[-20:]:
    doc_resp = requests.get(doc)
    doc_str = doc_resp.text
# Save the HTML link
    soup = BeautifulSoup(doc_str, 'html.parser')
    table_tag = soup.find('table', class_='tableFile', summary='Document Format Files')
    rows = table_tag.find_all('tr')
    cells = rows[1].find_all('td')
    html_link.append('https://www.sec.gov' + cells[2].a['href'])
# Save the SEC accession number (accnum)
    div_tag = soup.find('div', id='formHeader')
    secnum = div_tag.find('div', id='secNum')
    a = secnum.get_text().split()[3]
    accnum.append(a)
# Save the Filing Date and Reporting Period
    div_tag = soup.find('div', class_='formContent')
    dates = div_tag.find_all('div', class_='info')
    # Filing Date
    a = dates[0].get_text()
    fd.append(a)
    # Reporting Period
    b = dates[3].get_text()
    rp.append(b)
    # For 8K files, Save item info
    if obj_type == '8-K':
        c = dates[4].get_text()
        clist = re.findall(r'\d.\d\d', c)
        if clist != []:
            c = ', '.join(clist)
            item8k.append(c)
        else:
            clist = re.findall(r'\d', c)
            c = ', '.join(clist)
            item8k.append(c)
    else :
        c = 'NA'
        item8k.append(c)
# Save the Company name and CIK
    div_tag = soup.find('div', class_='companyInfo')
    comname = div_tag.find('span', class_='companyName')
    # Company Name
    a = comname.get_text().split("\n")[0]
    name.append(a)
    # CIK
    b = comname.get_text().split("\n")[1]
    cik.append(b)
# Save Business Address ZIP 
    div_tag = soup.find_all('div', class_='mailer')[1].find_all('span', class_='mailerAddress')[1]
    ba = div_tag.get_text()
    alist = re.findall(r'\d\d\d\d\d', ba)
    a = ', '.join(alist)
    bazip.append(a)
# Save SIC, File Type, Fiscal Year End and State of Incorporation
    div_tag = soup.find('div', class_='companyInfo')
    filinginfo = div_tag.find('p', class_='identInfo')
    # SIC
#   a = filinginfo.get_text().split("|")[5].split("SIC")[1].split()[1]
#   sic.append(a)
    # File Type
#   b = filinginfo.get_text().split("|")[2].split("Type")[1].split(":")[1]
#   filetype.append(b)
    # Fiscal Year End
    c = filinginfo.get_text().split("|")[2].split("Type")[0].split(":")[1]
    fye.append(c)
    # State
    d = filinginfo.get_text().split("|")[1].split(":")[1]
    state.append(d)
    
html_link = [w.replace('ix?doc=/', '') for w in html_link]
name = [w.replace(' (Filer)', '') for w in name]
cik = [w.replace('CIK: ', '') for w in cik]
cik = [w.replace(' (see all company filings)', '') for w in cik]

In [88]:
html_link

['https://www.sec.gov/Archives/edgar/data/95552/000156459020007814/sup-10k_20191231.htm',
 'https://www.sec.gov/Archives/edgar/data/95574/000143774920003101/sgc20191231_10k.htm',
 'https://www.sec.gov/Archives/edgar/data/95953/000009595320000021/synl-20191231x10k.htm',
 'https://www.sec.gov/Archives/edgar/data/96223/000009622320000008/jfg-2019113010k.htm',
 'https://www.sec.gov/Archives/edgar/data/96271/000156459020004873/ck0000096271-10k_20191231.htm',
 'https://www.sec.gov/Archives/edgar/data/9631/000089109220000862/e8102-10k.htm',
 'https://www.sec.gov/Archives/edgar/data/96869/000009686920000007/trc-2019123110k.htm',
 'https://www.sec.gov/Archives/edgar/data/96943/000009694320000017/tfx-20191231.htm',
 'https://www.sec.gov/Archives/edgar/data/97134/000009713420000008/form10-k12312019.htm',
 'https://www.sec.gov/Archives/edgar/data/97210/000119312520058676/d860886d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/97216/000009721620000027/tex201910-k.htm',
 'https://www.sec.gov/Arc

In [99]:
############### Create Data Frame
d = {'cik': cik, 'name': name, 'fd': fd, 'rp': rp, 'filetype': obj_type, 'item8k': item8k, 'accnum': accnum, 'bazip': bazip}
id_data = pd.DataFrame(data=d)
id_data

,cik,name,fd,rp,filetype,item8k,accnum,bazip
0,0000095552,SUPERIOR INDUSTRIES INTERNATIONAL INC,2020-02-28,2019-12-31,10-K,NA,0001564590-20-007814,48033
1,0000095574,"SUPERIOR GROUP OF COMPANIES, INC.",2020-02-20,2019-12-31,10-K,NA,0001437749-20-003101,33772
2,0000095953,SYNALLOY CORP,2020-03-06,2019-12-31,10-K,NA,0000095953-20-000021,
3,0000096223,Jefferies Financial Group Inc.,2020-01-29,2019-11-30,10-K,NA,0000096223-20-000008,10022
4,0000096271,TAMPA ELECTRIC CO,2020-02-18,2019-12-31,10-K,NA,0001564590-20-004873,
5,0000009631,BANK OF NOVA SCOTIA,2020-01-29,2019-10-31,10-K,NA,0000891092-20-000862,
6,0000096869,TEJON RANCH CO,2020-03-10,2019-12-31,10-K,NA,0000096869-20-000007,
7,0000096943,TELEFLEX INC,2020-02-21,2019-12-31,10-K,NA,0000096943-20-000017,
8,0000097134,TENNANT CO,2020-02-27,2019-12-31,10-K,NA,0000097134-20-000008,
9,0000097210,"TERADYNE, INC",2020-03-02,2019-12-31,10-K,NA,0001193125-20-058676,01864


In [17]:
############### Download HTML into TXT files
for link in html_link:
    urllib.request.urlretrieve(link, '..\\filings\\raw\\'+str(filenum[html_link.index(link)])+'.txt')